In [2]:
import pandas as pd
from tqdm import tqdm
import os

In [4]:
df_mapping_no2_to_traffico = pd.read_csv("../data_deliveries/4_no2_to_traffic_sensor_mapping.csv")

In [5]:
def process_traffic_data(df_traffic):
    """
    Función para procesar datos de tráfico a nivel horario a partir de datos cada 15 minutos.
    
    Se agrupan los datos por sensor y hora, calculando:
    - **Intensidad**: Promedio de la intensidad (vehículos/hora).
    - **Carga**: Promedio ponderado de la carga, usando la intensidad como peso.
    - **Ocupación**: Promedio ponderado de la ocupación, usando la intensidad como peso.
    - **Velocidad media (vmed)**: Promedio ponderado de la velocidad media, usando la intensidad como peso.

    Parámetros:
        df_traffic (pd.DataFrame): DataFrame con datos de tráfico a nivel de 15 minutos.

    Retorna:
        pd.DataFrame: DataFrame con los datos agregados a nivel horario.
    """

    # Convertir la columna 'fecha' a tipo datetime para manipulación de fechas
    df_traffic['fecha'] = pd.to_datetime(df_traffic['fecha'])

    # Redondear la fecha a la hora más cercana (para agrupar los datos por hora)
    df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')

    # Rellenar valores NaN con -1 para evitar problemas en cálculos posteriores
    #df_traffic = df_traffic.fillna(-1)

    # Función para calcular un promedio ponderado
    def weighted_average(group, value_column, weight_column):
        """
        Calcula la media ponderada de una columna usando otra como peso.

        Parámetros:
            group (pd.DataFrame): Grupo de datos a procesar.
            value_column (str): Nombre de la columna cuyo promedio ponderado se calcula.
            weight_column (str): Nombre de la columna que se usa como peso.

        Retorna:
            float: Media ponderada.
        """
        values = group[value_column]
        weights = group[weight_column]
        
        # Si la suma de los pesos es mayor a 0, se calcula la media ponderada, si no, se devuelve 0
        weighted_avg = (values * weights).sum() / weights.sum() if weights.sum() > 0 else 0
        return weighted_avg

    # Agrupar los datos por sensor y hora
    grouped = df_traffic.groupby(['id_trafico', 'hora'])

    # Lista para almacenar los resultados procesados
    result = []

    # Procesar cada grupo de sensor y hora
    for name, group in tqdm(grouped, desc="Processing groups"):
        """
        Para cada grupo (es decir, cada sensor en cada hora):
        - Se calcula la **intensidad** como promedio de la intensidad en los 4 intervalos de 15 minutos.
        - Se calcula la **carga** como un promedio ponderado de los valores de carga usando intensidad como peso.
        - Se calcula la **ocupación** como un promedio ponderado usando la intensidad como peso.
        - Se calcula la **velocidad media (vmed)** como un promedio ponderado usando la intensidad como peso.
        """

        # Intensidad: Como ya está en vehículos/hora, tomamos el promedio, no la suma
        intensidad_mean = group['intensidad'].mean()

        # Carga: Media ponderada con intensidad como peso
        carga_mean = weighted_average(group, 'carga', 'intensidad')

        # Ocupación: Media ponderada con intensidad como peso (no se puede sumar porque es un porcentaje)
        ocupacion_mean = weighted_average(group, 'ocupacion', 'intensidad')

        # Velocidad media: Media ponderada con intensidad como peso
        vmed_weighted = weighted_average(group, 'vmed', 'intensidad')

        # Agregar los resultados procesados a la lista
        result.append({
            'id_trafico': name[0],   # ID del sensor de tráfico
            'hora': name[1],         # Hora agregada
            'intensidad': intensidad_mean,  # Promedio de intensidad (veh/h)
            'carga': carga_mean,           # Promedio ponderado de carga
            'ocupacion': ocupacion_mean,   # Promedio ponderado de ocupación
            'vmed': vmed_weighted          # Promedio ponderado de velocidad media
        })

    # Convertir la lista de resultados en un DataFrame
    df_traffic = pd.DataFrame(result)

    return df_traffic  # Retornar el DataFrame procesado


In [6]:
df_mapping_no2_to_traffico['id_trafico'] = df_mapping_no2_to_traffico['id_trafico'].astype(int).astype(str)
filtered_traffic_sensors = list(df_mapping_no2_to_traffico.id_trafico.unique())

In [7]:
len(filtered_traffic_sensors)

71

In [8]:
# Define the root directory
root_dir = '../../data/raw/traffic'

# Iterate through all year folders
for year in range(2018, 2025):
    year_folder = os.path.join(root_dir, str(year))
    
    # Check if the year folder exists
    if os.path.isdir(year_folder):
        print(f"Processing folder: {year_folder}")
        
        # Iterate through files in the year folder
        for file_name in os.listdir(year_folder):
            if file_name.endswith('.csv'):  # Ensure it's a CSV file
                
                
                file_path = os.path.join(year_folder, file_name)
                
                try:
                    df = pd.read_csv(file_path, delimiter= ";")
                    
                    df = df.rename(columns = {'hora': 'fecha'})
                    df['fecha'] = pd.to_datetime(df['fecha'])
                        
                    # algunos se llaman id y otros identif
                    # Rename columns if they exist
                    if 'identif' in df.columns:
                        # for data before 2018...
                        df = df.rename(columns={'identif': 'id_trafico'})

                    if 'id' in df.columns:

                        df = df.rename(columns={'id': 'id_trafico'})
                        
                    df['id_trafico'] = df['id_trafico'].apply(lambda x: str(int(x)) if str(x).isdigit() else str(x))
                    df['id_trafico'] = df['id_trafico'].astype(str)

                    print("len df before filtering ", len(df))
                    print("Unique traffic sensors before filtering:" , df.id_trafico.nunique())

                    df = df[df['id_trafico'].isin(filtered_traffic_sensors)]
                    
                    print("len df after filtering ", len(df))
                    print("Unique traffic sensors after filtering:" , df.id_trafico.nunique())
                    
                    # if 'S' in df.error.unique():
                    #     print(f"There are errors in file: {file_name}")
                    #     print("There are a total of ", len(df[df['error'] == 'S']) , " errors")
                        
                    #     df = corregir_errores(df)

                    df = df[df['error'] == 'N']
                        
                    df = process_traffic_data(df)
                        
                    # Create the new file name
                    new_file_name = f"{file_name[:-4]}_processed.parquet"
                    new_file_path = os.path.join(year_folder, new_file_name)
                    new_file_path = new_file_path.replace("raw", "processed")
                    
                    # Create the processed directory if it does not exist
                    processed_dir = os.path.dirname(new_file_path)
                    os.makedirs(processed_dir, exist_ok=True)

                    # Save the processed DataFrame
                    df.to_parquet(new_file_path, index=False)
                    
                    print(f"Processed and saved: {new_file_path}")
                except Exception as e:
                    print(f"Error processing file {file_name}: {e}")


Processing folder: ../../data/raw/traffic/2018
entra id
len df before filtering  11551114
Unique traffic sensors before filtering: 3996
len df after filtering  158742
Unique traffic sensors after filtering: 55


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_56655/1748940065.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 39973/39973 [00:05<00:00, 6778.60it/s]


Processed and saved: ../../data/processed/traffic/2018/01-2018_processed.parquet
entra id
len df before filtering  11202244
Unique traffic sensors before filtering: 4022
len df after filtering  155102
Unique traffic sensors after filtering: 56


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_56655/1748940065.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 38895/38895 [00:05<00:00, 6800.16it/s]


Processed and saved: ../../data/processed/traffic/2018/06-2018_processed.parquet
entra id
len df before filtering  11392027
Unique traffic sensors before filtering: 4021
len df after filtering  159522
Unique traffic sensors after filtering: 56


...

Processed and saved: ../../data/processed/traffic/2024/12-2024_processed.parquet
entra id
len df before filtering  13357035
Unique traffic sensors before filtering: 4706
len df after filtering  184864
Unique traffic sensors after filtering: 64


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_56655/1748940065.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 46887/46887 [00:07<00:00, 6282.32it/s]


Processed and saved: ../../data/processed/traffic/2024/05-2024_processed.parquet
entra id
len df before filtering  13010780
Unique traffic sensors before filtering: 4695
len df after filtering  183015
Unique traffic sensors after filtering: 65


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_56655/1748940065.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 46183/46183 [00:07<00:00, 6317.98it/s]


Processed and saved: ../../data/processed/traffic/2024/04-2024_processed.parquet
